## Prepping data for testing


In [112]:
from __future__ import division
import numpy as np
from sklearn import datasets
import random
import pprint
import nltk
from operator import itemgetter
from gensim.models import Word2Vec 
from nltk.tokenize import word_tokenize
from sklearn.cluster import KMeans

In [2]:
wordvecs = Word2Vec.load_word2vec_format(
    "/Users/michael/Documents/Gaussian_LDA-master/data/glove.wiki/glove.6B.50d.txt", binary=False)
data = datasets.fetch_20newsgroups(remove=('headers', 'footers', 'quotes'), subset="train",
                                   categories=['comp.windows.x', 'sci.med',
                                               'talk.politics.mideast', 'soc.religion.christian',
                                               'sci.space'])

In [485]:
# data[0]['target_names']

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [3]:
vocab = set(wordvecs.vocab.keys())
stopwords = set(nltk.corpus.stopwords.words(fileids='english'))

In [4]:
txt = data['data']

In [5]:
len(txt)

2943

In [6]:
txt[100]

u''

In [506]:
most = 0
for doc in txt:
    if len(doc) > 2000: most += 1
print most

1138


In [71]:
plaindoc = "would one like to eat a banana but".split()

In [72]:
filter(lambda x: x != "would" and x != "one", plaindoc)

['like', 'to', 'eat', 'a', 'banana', 'but']

In [73]:
filter(lambda x: x not in stopwords, plaindoc)

['would', 'one', 'like', 'eat', 'banana']

In [74]:
plaindoc

['would', 'one', 'like', 'to', 'eat', 'a', 'banana', 'but']

In [91]:
cleandocs = []
for doc in txt:
    doc = doc.split()
#     doc = [word for word in doc if word in vocab]
#     doc = [word for word in doc if not word in stopwords]
#     doc = [word for word in doc if word.isalpha()]
    doc = filter(lambda x: x in vocab, doc)
    doc = filter(lambda x: x not in stopwords, doc)
    doc = filter(lambda x: x.isalpha(), doc)
    doc = filter(lambda x: x != "would" and x != "one" and x !='like' and x !='even', doc)
    doc = ' '.join(doc)
    cleandocs.append(doc)

In [92]:
len(cleandocs)

2943

In [93]:
cleandocs = [doc for doc in cleandocs if len(doc) < 1500]

In [94]:
0 % 20 ==0

True

In [95]:
words = []
for doc in cleandocs:
    for word in doc.split():
        words.append(word)

In [96]:
for word in words:
    if word =='even':
        print 'yes'

In [97]:
fd = nltk.probability.FreqDist(words)

In [98]:
fd.most_common(n=10)

[(u'people', 690),
 (u'know', 610),
 (u'get', 608),
 (u'think', 566),
 (u'also', 494),
 (u'could', 458),
 (u'use', 430),
 (u'may', 426),
 (u'see', 386),
 (u'much', 365)]

In [99]:
bb = itemgetter(*np.random.random_integers(0, 1650, 600).tolist())
a  = bb(cleandocs)

In [100]:
a = list(a)

In [101]:
a.insert(0, test_doc)

In [102]:
test_doc = "hockey basketball baseball football sports touchdown golf playoffs " * 1000
a.append(test_doc)

In [103]:
test_doc.startswith("hockey basketball baseball football sports touchdown golf playoffs ")

True

In [104]:
bb = itemgetter(*np.random.random_integers(0, 1650, 500).tolist())
len(bb(cleandocs))

f = '/Users/michael/Documents/GaussianLDA/clean20news.txt'
# np.savetxt(f, docs)
with open(f, 'w') as fi:
    for doc in a:
        try:
            fi.write("%s\n" % doc)
        except UnicodeEncodeError: continue

#### Testing to make sure we can read it well

In [527]:
# f = '/Users/michael/Documents/GaussianLDA/clean20news.txt'
# with open(f, 'r') as fi:
#     text = fi.read().splitlines()
#     fi.close()
# print len(text)

In [106]:
a, b, c = [0,0], [0,0], [0,0]

for x, y in zip(txt, data['target']): 
    if y == 0: a[0] += len(x); a[1] += 1
    if y == 1: b[0] += len(x); b[1] += 1
    if y == 2: c[0] += len(x); c[1] += 1
        
print a[0] / a[1], a[1]
print b[0] / b[1], b[1]
print c[0] / c[1], c[1]

In [404]:
# mansour_topic = np.loadtxt("/Users/michael/Documents/GaussianLDA/output/1iter4topic2Topic Mean.txt")
# das_topic = np.loadtxt("/Users/michael/Documents/Gaussian_LDA-master/output/Aliaswiki50D20T3.txt")

In [447]:
vecs = []
vocab = set([])
for doc in cleandocs:
    for word in doc.split():
        vocab.add(word)
        
for word in vocab:
    try: vecs.append(wordvecs[word])
    except: None


In [450]:
km = KMeans(n_clusters=15, n_jobs=-1)
km.fit(np.array(vecs))
wordvecs.most_similar(positive=[km.cluster_centers_[3]])

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=15, n_init=10,
    n_jobs=-1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

## Making More Sythetic Data

In [128]:
%pprint on
topic1 = ' '.join([word for word, score in wordvecs.most_similar(positive=['car'], topn=20)])
topic2 = ' '.join([word for word, score in wordvecs.most_similar(positive=['science'], topn=20)])
topic3 = ' '.join([word for word, score in wordvecs.most_similar(positive=['money'], topn=20)])
print topic1, topic2, topic3

Pretty printing has been turned ON
truck cars vehicle driver driving bus vehicles parked motorcycle taxi passenger pickup trucks cab suv train drivers bicycle jeep airplane sciences research institute studies physics psychology scientific biology studying chemistry study economics mathematics journalism university engineering professor humanities sociology philosophy cash paying funds pay raise paid billions millions get fund keep tax savings credit make putting taxes spend making giving


In [129]:
{word:score for word, score in wordvecs.most_similar(positive=['car'])}

{u'bus': 0.8210511207580566,
 u'cars': 0.8870189189910889,
 u'driver': 0.8464018702507019,
 u'driving': 0.8384189009666443,
 u'motorcycle': 0.7866503000259399,
 u'parked': 0.7902189493179321,
 u'taxi': 0.7833929061889648,
 u'truck': 0.92085862159729,
 u'vehicle': 0.8833683729171753,
 u'vehicles': 0.8174992799758911}

In [130]:
docs = [topic1, topic2, topic3]

f = '/Users/michael/Documents/GaussianLDA/clean20news.txt'
# np.savetxt(f, docs)
with open(f, 'w') as fi:
    for doc in docs:
        try:
            fi.write("%s\n" % doc)
        except UnicodeEncodeError: continue

Current project:

you have unlabeled cans of paint.  You want to be able to assign color names to them.

My model currently just dumps all the cans of paint into a bucket, mixes it up, and labels each paint can that gross dark brown purlply color that comes out when a child mixes all the paints together. 

In [131]:
#Testing kmeans cluster centroids

topic1 = [word for word, score in wordvecs.most_similar(positive=['car'], topn=20)]
topic2 = [word for word, score in wordvecs.most_similar(positive=['science'], topn=20)]
topic3 = [word for word, score in wordvecs.most_similar(positive=['money'], topn=20)]

In [134]:
arrs = []
for doc in [topic1, topic2, topic3]:
    for word in doc:
        arrs.append(wordvecs[word])
arrs = np.array(arrs)

In [140]:
k = 3
km = KMeans(n_clusters=k)

In [141]:
km.fit(arrs)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [142]:
for i in range(k):
    print wordvecs.most_similar(positive=[km.cluster_centers_[i]])

[(u'money', 0.95746248960495), (u'pay', 0.9299221038818359), (u'raise', 0.9211190342903137), (u'paying', 0.9172735810279846), (u'cash', 0.9055809378623962), (u'cost', 0.8760908246040344), (u'funds', 0.8710236549377441), (u'giving', 0.8627578020095825), (u'paid', 0.861911416053772), (u'costs', 0.8605511784553528)]
[(u'science', 0.9424635171890259), (u'studies', 0.920809268951416), (u'psychology', 0.9107574224472046), (u'sciences', 0.9034210443496704), (u'biology', 0.8869606256484985), (u'mathematics', 0.8854325413703918), (u'physics', 0.8730563521385193), (u'institute', 0.8722023963928223), (u'studying', 0.8703454732894897), (u'anthropology', 0.8632563352584839)]
[(u'car', 0.9455156326293945), (u'truck', 0.9394636750221252), (u'vehicle', 0.9090796709060669), (u'cars', 0.9007307291030884), (u'trucks', 0.8714910745620728), (u'vehicles', 0.8700135350227356), (u'bus', 0.8697482943534851), (u'taxi', 0.8601557016372681), (u'passenger', 0.851845383644104), (u'cab', 0.8493589162826538)]


## TODO:

Start looking at the pdf's of each word and make sure they're being assigned to the right topics... even though the actual topics might be wrong, lets make sure that it is fairly consistant. 

set a break point for certain intervals, and use the debugger console to check the values. 